# 10.1

In [ ]:
# 최근 영업일 데이터 뽑기

import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text   # select_one - 해당 데이터 태그 추출

print(parse_day)

In [ ]:
import re

biz_day = re.findall('[0-9]+', parse_day)   # 숫자에 해당하는 부분 추출
biz_day = ''.join(biz_day)  # 숫자 합쳐주기

print(biz_day)

## 10.2 한국거래소 업종분류 현황 및 개별지표 크롤링

## 10.2.1

In [ ]:
# 1. http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201에서 CSV 데이터를 받아오기 위한 OTP 받아오기

import requests as rq
import cloudscraper     # 보안서비스 우회 요청 처리 라이브러리
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'locale':'ko-KR',
    'mktId': 'STK', # 코스피 종목
    'trdDd': biz_day,
    'money':'1',
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03901'
}
# headers = {'Referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201'}   # Referer: 링크를 통해 각각의 웹사이트로 방문할 때 남는 흔적, 이 흔적이 없으면 서버가 로봇으로 인식해 정보를 주지 않음

scraper = cloudscraper.create_scraper()

response = scraper.post(gen_otp_url, params=gen_otp_stk)    # 포스트 요청 - cloudscraper 사용하여 요청 우회

otp_stk = response.text

print(otp_stk)

In [ ]:
# 위의 과정을 거처 생성된 OTP를 제출 시, 원하는 데이터 다운 가능

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = scraper.post(down_url, params={'code': otp_stk})              # cloudscraper 사용
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')   # BytesIO : 받은 데이터의 content 부분을 바이너리 스트림으로 만든 후 read_csv 함수를 통해 데이터 읽어오기

sector_stk.head()

In [ ]:
# 코스닥 시작 데이터도 동일 과정을 통해 다운

gen_otp_ksq = {
    'locale':'ko-KR',
    'mktId': 'KSQ', # 코스피 종목
    'trdDd': biz_day,
    'money':'1',
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03901'
}

otp_ksq = scraper.post(gen_otp_url, params=gen_otp_ksq).text

down_sector_ksq = scraper.post(down_url, params={'code': otp_ksq})
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

In [ ]:
# 코스피, 코스닥 데이터 합치기

krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)     # 두 데이터 합쳐주고 reset_index()로 인덱스 리셋, drop=True로 인덱스로 세팅한 열을 삭제
krx_sector['종목명'] = krx_sector['종목명'].str.strip()     # 종목명에 공백이 있는 경우 삭제
krx_sector['기준일'] = biz_day

krx_sector.head()

## 10.2.2

In [ ]:
# 개별지표 OTP 생성 후 데이터 불러오기

import cloudscraper     # 보안서비스 우회 요청 처리 라이브러리
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType':'1',
    'mktId':'ALL',
    'trdDd':biz_day,
    'csvxls_isNo':'false',
    'name':'fileDown',
    'url':'dbms/MDC/STAT/standard/MDCSTAT03501'
}
# headers={'Referer':'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
scraper = cloudscraper.create_scraper()
otp = scraper.post(gen_otp_url, params=gen_otp_data).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = scraper.post(down_url, params={'code': otp})

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()   # 종목명 없는 항목은 삭제